# 1) Load data

In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data = pd.read_csv(r'C:\Users\eelis\listings.csv')

c:\users\eelis\python3.6.1\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Put columns into list for future inspection
ols = data.columns.tolist()

# 2) Cleaning data

Choosing the columns will take some time and be one of the most time-consuming parts. So far, I have discareded columns that a)dont make sense for the prediction (like urls, id's etc) b)have too many nans c)have too many different categories (leads to a explosion in number of columns)

In [4]:
#---------------------REVIEW : SEE IF WANT TO KEEP SOME COLUMNS
data = pd.read_csv(r'C:\Users\eelis\listings.csv') #Change to your own
data = data.drop(['square_feet','host_acceptance_rate','jurisdiction_names','license','experiences_offered','id', 'listing_url','scrape_id','last_scraped','name','summary','space'
                 ,'description','neighborhood_overview','notes','transit','access','interaction','house_rules'
                 ,'thumbnail_url','medium_url','picture_url','xl_picture_url',
                 'host_url','host_id','host_about','host_thumbnail_url','host_picture_url','state','country',
                 'country','host_name'], axis=1)

In [5]:
#Inspect the first 5 rows
#--------------TO REVIEW: See if some features do not make sense
data.head()

,host_since,host_location,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,zipcode,market,smart_location,country_code,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2008-11-25,"New York, New York, United States",a few days or more,0%,f,Harlem,1.0,1.0,"['email', 'phone', 'google', 'reviews', 'jumio...",t,t,"New York, NY, United States",Harlem,Harlem,Manhattan,New York,10027,New York,"New York, NY",US,40.80902,-73.94190,t,Apartment,Private room,2,1.0,1.0,1.0,Pull-out Sofa,"{""Cable TV"",Internet,Wifi,""Air conditioning"",K...",$150.00,NaN,NaN,$200.00,$75.00,2,$20.00,3,7,3,3,7,7,3.0,7.0,37 months ago,t,30,60,90,365,2019-09-13,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f,f,strict_14_with_grace_period,t,t,1,0,1,0,NaN
1,2008-12-07,"New York, New York, United States",within an hour,92%,f,Clinton Hill,1.0,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Brooklyn, NY, United States",Brooklyn,Clinton Hill,Brooklyn,Brooklyn,11238,New York,"Brooklyn, NY",US,40.68514,-73.95976,t,Guest suite,Entire home/apt,3,1.0,1.0,4.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$89.00,$575.00,"$2,100.00",$500.00,NaN,1,$0.00,1,730,1,1,730,730,1.0,730.0,3 days ago,t,1,3,8,192,2019-09-13,279,67,2014-09-30,2019-08-29,90.0,9.0,9.0,10.0,9.0,10.0,9.0,f,f,f,moderate,f,f,1,1,0,0,4.62
2,2009-01-29,"Miami, Florida, United States",NaN,NaN,f,East Harlem,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"New York, NY, United States",East Harlem,East Harlem,Manhattan,New York,10029,New York,"New York, NY",US,40.79851,-73.94399,t,Apartment,Entire home/apt,1,1.0,NaN,1.0,Real Bed,"{Internet,Wifi,""Air conditioning"",Kitchen,Elev...",$80.00,$600.00,"$1,600.00",$100.00,$80.00,1,$20.00,10,120,10,10,120,120,10.0,120.0,6 months ago,t,0,0,0,0,2019-09-13,9,2,2012-03-20,2018-11-19,93.0,10.0,9.0,10.0,10.0,9.0,10.0,f,f,f,strict_14_with_grace_period,t,t,1,1,0,0,0.10
3,2009-02-02,"New York, New York, United States",within a day,78%,f,Flatiron District,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,f,"New York, NY, United States",Midtown East,Murray Hill,Manhattan,New York,10016,New York,"New York, NY",US,40.74767,-73.97500,f,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Buzzer/w...",$200.00,NaN,NaN,$300.00,$125.00,2,$100.00,3,21,3,3,21,21,3.0,21.0,today,t,3,3,13,13,2019-09-13,75,10,2009-04-20,2019-07-21,89.0,10.0,9.0,10.0,10.0,9.0,9.0,f,f,f,moderate,t,t,1,1,0,0,0.59
4,2009-02-03,"New York, New York, United States",a few days or more,0%,f,Bedford-Stuyvesant,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'off...",t,f,"Brooklyn, NY, United States",Bedford-Stuyvesant,Bedford-Stuyvesant,Brooklyn,Brooklyn,11216,New York,"Brooklyn, NY",US,40.68688,-73.95596,f,Apartment,Private room,2,NaN,1.0,1.0,Futon,"

In [6]:
#ISSUE: Some data is from places like LA etc for many features which we need to remove!!
data.host_location.value_counts().head()

New York, New York, United States         31537
US                                         5265
Brooklyn, New York, United States          3744
Queens, New York, United States            1343
Los Angeles, California, United States      282
Name: host_location, dtype: int64

In [7]:
#Same story for this variable
#REVIEW: Check case for other features as well - just looked at two (should take care of majority tbh)
data.market.value_counts().head()

New York                       48245
Other (Domestic)                  15
Adirondacks                        2
Paris                              1
Catskills and Hudson Valley        1
Name: market, dtype: int64

In [8]:
#Create list of 5 Boroughs to use as a filter
areaList = ['New York, New York, United States','Brooklyn, New York, United States','Queens, New York, United States',
            'Bronx, New York, United States','Staten Island, New York, United States']

In [9]:
data = data[data['market'] =='New York']
data = data[data['host_location'].isin(areaList)]

In [10]:
#Check that filter worked
data.host_location.value_counts()

New York, New York, United States         31432
Brooklyn, New York, United States          3738
Queens, New York, United States            1339
Bronx, New York, United States              259
Staten Island, New York, United States      124
Name: host_location, dtype: int64

# Clean price data

In [12]:
#Convert from dollars to float
data['price'] = data['price'].str.replace('$', '')
data['price'] = data['price'].str.replace(',', '')
data['price'] = data['price'].astype(float)

# One-hot code string data into int + vector

In [13]:
#Look at number of unique values in each string column
#If there are too many distinct values, say more than 10, do not include it (explosion in columns otherwise)
strColumn = data.select_dtypes(include='object')
columnDrop = []
for name in strColumn:
    print(name,data[name].nunique())
    if data[name].nunique() > 10:
        columnDrop.append(name)
data = data.drop(columns=columnDrop,axis = 1)

host_since 3602
host_location 5
host_response_time 4
host_response_rate 86
host_is_superhost 2
host_neighbourhood 297
host_verifications 488
host_has_profile_pic 2
host_identity_verified 2
street 263
neighbourhood 192
neighbourhood_cleansed 219
neighbourhood_group_cleansed 5
city 257
zipcode 209
market 1
smart_location 263
country_code 1
is_location_exact 2
property_type 31
room_type 4
bed_type 5
amenities 34097
weekly_price 563
monthly_price 595
security_deposit 192
cleaning_fee 190
extra_people 96
calendar_updated 90
has_availability 1
calendar_last_scraped 3
first_review 2901
last_review 1766
requires_license 1
instant_bookable 2
is_business_travel_ready 1
cancellation_policy 6
require_guest_profile_picture 2
require_guest_phone_verification 2


In [14]:
#Get list of columns with string data that we need to convert to int
#We then seperate each category into a different column.
#REVIEW :Need to make a call on how many text columns we can have since number of columns will explode fast
strColumn = data.select_dtypes(include='object')
for name in strColumn:
    print(name)
    one_hot = pd.get_dummies(data[name],prefix = name)  #Add original feature name as prefix
    # Drop column B as it is now encoded
    data = data.drop(name,axis = 1)
    # Join the encoded df
    data = data.join(one_hot)

host_location
host_response_time
host_is_superhost
host_has_profile_pic
host_identity_verified
neighbourhood_group_cleansed
market
country_code
is_location_exact
room_type
bed_type
has_availability
calendar_last_scraped
requires_license
instant_bookable
is_business_travel_ready
cancellation_policy
require_guest_profile_picture
require_guest_phone_verification


In [16]:
#See remaining nan -->Square feet has little so dont use those
data.isnull().sum().sort_values(ascending=False)

review_scores_location                                  6861
review_scores_value                                     6860
review_scores_checkin                                   6857
review_scores_accuracy                                  6845
review_scores_communication                             6841
review_scores_cleanliness                               6831
review_scores_rating                                    6816
reviews_per_month                                       6205
bathrooms                                                 40
bedrooms                                                  29
beds                                                      25
require_guest_phone_verification_t                         0
calculated_host_listings_count                             0
calculated_host_listings_count_shared_rooms                0
calculated_host_listings_count_entire_homes                0
calculated_host_listings_count_private_rooms               0
number_of_reviews       

# 3) Apply machine learning (softly)

In [17]:
data = data.dropna(axis='rows')  #REVIEW THIS DECISION - CAN IMPROVE THIS BY CALCULATING AVERAGES etc

X = data.drop(['price'], axis=1)
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
model.score(X_test, y_test)

0.4140174204673966